# R on Vertex AI Workbench Instances

How to use R on a Vertex AI Workbench Instance

## Setup

1. Create a new [Vertex AI Workbench Instance](https://cloud.google.com/vertex-ai/docs/workbench/instances/introduction) via the console or gcloud CLI below
    * documentation: https://cloud.google.com/vertex-ai/docs/workbench/instances/create
    * gcloud: https://cloud.google.com/sdk/gcloud/reference/workbench/instances

```sh
gcloud workbench instances create r-instance-01 --location=us-central1-c --machine-type=n2-standard-8
```

2.  Open a terminal session from the Launcher     
3.  enter commands below in the terminal (one line at a time)
 
```sh
ENV_NAME=r-env
conda create -n $ENV_NAME r-essentials r-base rpy2 --yes
conda activate $ENV_NAME
python -m ipykernel install --user --name=$ENV_NAME
```
  
4.  Return to the Launcher screen and click the R kernel (if it does not appear, reset the instance from the console and try again)   
5.  To try out a few R code examples, clone this repo (below), upload this individual notebook file to your instance or copy/paste!

```sh
git clone https://github.com/justinjm/vertex-ai-examples.git && cd vertex-ai-examples/98-r-examples
```

## Install R packages

In [ ]:
required_packages <- c("bigrquery", "googleCloudStorageR")
install.packages(setdiff(required_packages, rownames(installed.packages())))

## set constants 

### Email & Project ID

In [ ]:
email <- "your-email@your-company-name.com"
project_id <- "your-project-id"
## try to set from gcloud if not specified 
if (email == "your-email@your-company-name.com") {
    email <- system("gcloud config get-value core/account", intern = TRUE)}
if (project_id == "your-project-id"){
    project_id <- system("gcloud config get-value project", intern = TRUE)}

In [ ]:
print(email)

In [ ]:
print(project_id)

### Other

In [ ]:
experiment_name <- "r-kernel-workbench"

bucket_name <- paste0(project_id, "-" , experiment_name)

dataset_name <- gsub("-", "_", experiment_name)
table_name <- "californiahousing"

location <- "us-central1"

data_filename <- "california-housing-tabular-regression.csv"
data_uri <- "gs://cloud-samples-data/ai-platform-unified/datasets/tabular/california-housing-tabular-regression.csv"

## load R packages 

In [ ]:
library(googleCloudStorageR)
library(bigrquery)
library(gargle)
# set when debugging
# options(gargle_verbosity = "debug") 

##  Clients

Set scope to enable access for each R package and use token based auth and to simplify overall process

Refs:

* GCS - https://code.markedmondson.me/googleCloudStorageR/articles/googleCloudStorageR.html#token-authentication-1
* BQ - https://bigrquery.r-dbi.org/reference/bq_auth.html

In [ ]:
scope <- c("https://www.googleapis.com/auth/cloud-platform")
token <- token_fetch(scopes = scope, email = email)

In [ ]:
## authenticate with each service
gcs_auth(token = token)
bq_auth(token = token)

## Test connections (optional)

### List Google Cloud Storage Buckets

In [ ]:
# buckets <- gcs_list_buckets(project_id)
# buckets

### List Bigquery datasets 

In [ ]:
# datasets <- as.data.frame(do.call(rbind, bq_project_datasets(project_id)))
# datasets

## Example workflow

* get public sample data (CSV) file from public GCS bucket
* Create GCS staging bucket and destination BQ table
* load data from GCS to BQ
* query BQ data
* cleanup / delete all created resources

## create GCS Bucket 

In [ ]:
create_bucket_if_not_exists <- function(project_id, bucket_name, location) {
    # Check if the bucket already exists by trying to retrieve it
    bucket_exists <- tryCatch({
      gcs_get_bucket(bucket_name)
      TRUE  # If no error, the bucket exists
    }, error = function(e) {
      message(paste("Bucket", bucket_name, "does not exist."))
      FALSE # If an error is thrown, the bucket doesn't exist
    })

    if (!bucket_exists) {
      # Create the bucket if it doesn't exist
      gcs_create_bucket(bucket_name,
                        projectId = project_id,
                        storageClass = "STANDARD",
                        location = location)
      message(paste("Bucket", bucket_name, "created successfully."))
    } else {
      message(paste("Bucket", bucket_name, "already exists."))
    }
}

In [ ]:
create_bucket_if_not_exists(project_id = project_id, 
                            bucket_name = bucket_name,
                            location = location)

In [ ]:
create_dataset_if_not_exists <- function(project_id, dataset_id, location) {
    
    # Check if the dataset already exists
    dataset_exists <- bq_dataset_exists(bq_dataset(project_id, dataset_id))  # Retrieve dataset information

  if (dataset_exists!=TRUE) {
    # Create the dataset if it doesn't exist
    bq_dataset_create(bq_dataset(project_id, dataset_id), location)
    message(paste("Dataset", dataset_id, "created successfully in project", project_id))
  } else {
    message(paste("Dataset", dataset_id, "already exists in project", project_id))
  }
}

In [ ]:
create_dataset_if_not_exists(project_id = project_id, 
                             dataset_id = dataset_name, 
                             location = location)

In [ ]:
## set global bucket first so we don't need to in future api calls
gcs_global_bucket(bucket_name)
gcs_list_objects()


In [ ]:
## download file from bucket ------------------------------------------------
data_raw <- gcs_get_object(object_name = data_uri)

In [ ]:
## inspect data to sanity check ---------------------------------------------
summary(data_raw)
head(data_raw)

## upload file to our bucket

In [ ]:
gcs_upload(data_raw,
           name = data_filename,
           predefinedAcl = "bucketLevel")

In [ ]:
## list to confirm upload  -----------------------------------------------
gcs_list_objects()

## Create new empty BQ table to ingest data from GCS

In [ ]:
### name of bq table to create, save as object -----------------------------
bq_table_to_create <- bq_table(project_id, dataset_name, table_name)
bq_table_to_create

In [ ]:
### set fields / schema -----------------------------------------------------
bq_fields_to_create <- as_bq_fields(
  list(
    list(name = "longitude", type = "STRING"),
    list(name = "latitude", type = "STRING"),
    list(name = "housing_median_age", type = "STRING"),
    list(name = "total_rooms", type = "STRING"),
    list(name = "total_bedrooms", type = "STRING"),
    list(name = "population", type = "STRING"),
    list(name = "households", type = "STRING"),
    list(name = "median_income", type = "STRING"),
    list(name = "median_house_value", type = "STRING")
    )
  )


In [ ]:
## execute create table -----------------------------------------------------
bq_table_create(bq_table_to_create,
                bq_fields_to_create)

## Ingest data to BQ table

In [ ]:
bq_table_load(bq_table_to_create,
              source_uris = sprintf("gs://%s/%s",
                                    bucket_name,
                                    data_filename),
              source_format = "CSV",
              nskip = 1,
              create_disposition = "CREATE_IF_NEEDED",
              write_disposition = "WRITE_TRUNCATE")

Query the table to confirm creation:

In [ ]:
query <- sprintf("SELECT * FROM `%s.%s.%s` LIMIT 100",
                 project_id, dataset_name, table_name)


In [ ]:
bq_query_results <- bq_project_query(project_id, query)

In [ ]:
bq_data <- bq_table_download(bq_query_results)

In [ ]:
head(bq_data)

## cleanup

Change `delete_flag` to TRUE to permenantly delete GCS and BQ resources you just created.

In [ ]:
delete_flag = FALSE

### BQ - delete dataset and all underlying tables

In [ ]:
if (delete_flag == TRUE) {
    bq_dataset_delete(bq_dataset(project_id, dataset_name), delete_contents = TRUE)
}

### GCS - Delete bucket and all objects within

In [ ]:
if (delete_flag == TRUE) {
    gcs_delete_bucket(bucket_name, force_delete = TRUE)
}

### Workbench Instance

Then delete the Workbench Instance you're currently using via the console or via gcloud:

```sh
gcloud workbench instances delete r-instance-01 --location=us-central1-c
```

https://cloud.google.com/sdk/gcloud/reference/workbench/instances/delete